# Homework 6

Solution to exercises 2, 3, 4, 5 and 6 about Overfitting, Regularization and Weight Decay. Data for [training](http://work.caltech.edu/data/in.dta) and [testing](http://work.caltech.edu/data/out.dta) taken from external website of the Caltech course. Initial functions to import required libraries and load data from mentioned sources. Special thanks for the inspiration given from a similar [solution](http://lionoso.org/learningfromdata/Exercise2-Overfitting.py)

In [1]:
import pandas as pd
import numpy as np

train_data_str = pd.read_csv("http://work.caltech.edu/data/in.dta", header=None)
test_data_str = pd.read_csv("http://work.caltech.edu/data/out.dta", header=None)

#Transform train data from string to numeric
train_data_str = list(map(str.split,list(map(str.strip, np.squeeze(train_data_str.values)))))
tr_data = []
for tds in train_data_str:
    tr_data.append([float(i) for i in tds])

Y_train = [row[2] for row in tr_data]

#Transform test data from string to numeric
test_data_str = list(map(str.split,list(map(str.strip, np.squeeze(test_data_str.values)))))
tst_data = []
for tds in test_data_str:
    tst_data.append([float(i) for i in tds])

Y_test = [row[2] for row in tst_data]

Definition of the non linear transformation in a function

In [2]:
def transform(data):
    #Format data in a better way to perform transformation    
    tmp_data = []
    tmp_data = [((row[0], row[1]),(row[2])) for row in data]
    
    result = [(1, row[0][0], row[0][1], np.power(row[0][0], 2),
               np.power(row[0][1], 2), row[0][0]*row[0][1], 
               np.absolute(row[0][0] - row[0][1]), np.absolute(row[0][0] + row[0][1])) for row in tmp_data]
    return result

Definition of methods to compute the error. The error defined here counts the number of misclassified points and divides it by the total number of points.

In [3]:
def evaluate_g(ex,w):
    r = sum(ex_i*w_i for ex_i,w_i in zip(ex,w))
    if r > 0:
        return float(1)
    elif r < 0:
        return float(-1)
    
def calculate_error(X, w, y):
    n_wrong = 0
    for i in range(len(X)):
        if (y[i] != evaluate_g(X[i],w)):
            n_wrong += 1
    p_wrong = float(n_wrong) / len(X)
    return p_wrong

Making use of the defined transformation methods and error computation the linear regression is computed and in sample and out sample error are obtained.

In [4]:
trans_train = transform(tr_data)
trans_test = transform(tst_data)

#Compute the pseudo-inverse
inv_trans_train = np.linalg.pinv(trans_train)

#As a result we obtain the optimized weights
w = np.dot(inv_trans_train, Y_train)

in_sample_error = calculate_error(trans_train, w, Y_train)
out_sample_error = calculate_error(trans_test, w, Y_test)

print("In Sample Error: ", in_sample_error)
print("Out of Sample Error: ", out_sample_error)

In Sample Error:  0.02857142857142857
Out of Sample Error:  0.084


Definition of weight decay for the linear regression

In [5]:
def linear_regression_with_weight_decay(X, k, y):
    reg = np.identity(len(X[0])) * np.power(10,k)
    X = np.matrix(X)
    y = np.matrix(y)
    X_t = np.transpose(X)
    w = (np.linalg.inv( (X_t*X + reg) ) * X_t )*np.transpose(y)
    return w

Making use of the weight decay function for the range -3 to 3 which includes the required answers to the purposed questions

In [6]:
#k=-3

#w_decay = linear_regression_with_weight_decay(trans_train, k, Y_train)

#in_sample_error = calculate_error(trans_train, w_decay, Y_train)
#out_sample_error = calculate_error(trans_test, w_decay, Y_test)

#print("In Sample Error with Weight Decay: ", in_sample_error)
#print("Out of Sample Error with Weight Decay: ", out_sample_error)

for k_i in range (-3,4):
    w_decay = linear_regression_with_weight_decay(trans_train, k_i, Y_train)

    in_sample_error = calculate_error(trans_train, w_decay, Y_train)
    out_sample_error = calculate_error(trans_test, w_decay, Y_test)

    print("In Sample Error with Weight Decay for k=",k_i,", ", in_sample_error)
    print("Out of Sample Error with Weight Decay for k=",k_i,", ", out_sample_error)


In Sample Error with Weight Decay for k= -3 ,  0.02857142857142857
Out of Sample Error with Weight Decay for k= -3 ,  0.084
In Sample Error with Weight Decay for k= -2 ,  0.02857142857142857
Out of Sample Error with Weight Decay for k= -2 ,  0.084
In Sample Error with Weight Decay for k= -1 ,  0.02857142857142857
Out of Sample Error with Weight Decay for k= -1 ,  0.084
In Sample Error with Weight Decay for k= 0 ,  0.0
Out of Sample Error with Weight Decay for k= 0 ,  0.092
In Sample Error with Weight Decay for k= 1 ,  0.05714285714285714
Out of Sample Error with Weight Decay for k= 1 ,  0.124
In Sample Error with Weight Decay for k= 2 ,  0.2
Out of Sample Error with Weight Decay for k= 2 ,  0.228
In Sample Error with Weight Decay for k= 3 ,  0.37142857142857144
Out of Sample Error with Weight Decay for k= 3 ,  0.436


We can see that the out of sample and in sample error converge for K lower than -1, getting no variation for integer values lower than that. **Out of sample error = 0.084**